# The battle of Neighborhoods

## Introduction

According to the indroduction of the project, here comes an idea in my mind, which is opening an exotic restaruant in a certain city based on my orientation of hometown. 

Even if London is already an international city, with a lot of Asian food have already served in several locations here. However, as I chose to study master in United Kingdom for next 2 years, the homesickness of food and taste of cuisines always catch my attraction while walking on the street of cities. So I would like to research the suitable district to open a **Taiwanese flavored restaurant** in **London, United Kingdom** with several combinations from East Asian and North-east Asian to help the people who have the same background like me who wish to find a good place to dine in.

## Business Problem Description

Which neighborhoods should be the best decision of opening a Taiwanese restaurant in **London, United Kingdom**? How to choose the most suitable places?

## How to decide ?

1. Locate and find the data of neighborhoods in City of London or suburban area of Greater London, but better in city center of London(including postal code, just like what we did as Toronto City in the last assignment)
2. Discover and construct the datas of similar restaurants in London
3. Kmeans and clustering to find the venues 
4. Analyse the results and give some suggestions of locations which suit for the purpose of opening a Taiwanese restaurants in London


## Data

To explore, we need to import and build neighborhoods data of London from the most common data resources, wikiwpedia and foursquare.
 
First of all, we use the data, List of London Areas of boroughs: https://en.wikipedia.org/wiki/List_of_London_boroughs to create the datasets using what we learned before, web scrapping:

In [18]:
import numpy as np 
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim 
import folium

In [19]:
#Web scrapping from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
wikipedia_page = requests.get(url)
wikipedia_page

<Response [200]>

In [20]:
#Check the dataset first
df_london=pd.read_html(url, header=0)[0]

df_london.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham[note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [21]:
# After checking the dataset, we find some of the data are missing so we need to clear and replace to what we want
df_la = df_london.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
df_la['Inner'].replace(np.nan,'0', inplace=True)
df_la['Borough'].replace('Barking and Dagenham[note 1]','Barking and Dagenham', inplace=True)
df_la['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
df_la['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
df_la.head()
df_la['Inner'] = '0'
df_la.head()

,Borough,Inner,Area (sq mi),Population (2019 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp..."
1,Barnet,0,33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


After reset and clean the data, we should now rename the column names to make the datasets easier to recognise:

In [22]:
df_la['Inner'] = df_la.Borough.isin(Inn).astype(int)
Out = df_la[df_la.Inner == 0]

In [23]:
Out = Out.drop(['Inner'], axis=1)
df = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2019 est)[1]":"Population"})

In [24]:
geolocator = Nominatim(user_agent="London_explorer")

In [25]:
df['Co-ordinates']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [26]:
df[['Latitude', 'Longitude']] = df['Co-ordinates'].apply(pd.Series)

In [27]:
#After series of drop columns and create latitude and longitude, we can find the datasets like this:
df.head()

,Borough,Area,Population,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814


So the dataset df is the data formats for further investigating week2 Capstone project combination with Foursquare API which is saved for the web data

## How to solve the problem with datasets?

The most straightforward solution is to find the venues with the neighbourhoods of London Boroughs. Foursquare API would be considered of the assistance tools, however the number of venues investigated by FourSquare API will be limited. The neighborhoods to transport connection and other amenities would be correlated, at the same time,  accessibility and ease of supplies are also the priorty selection.